In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast


from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.feature_selection import f_regression


df = pd.read_csv(r"C:\Users\jjap2\Poleto SCE\Poleto-SCE\dond_game_data.csv")

In [24]:
#cleaning cell - selected via previous F-Test
features = ["Board Average","Previous Offer","Probability of Big Value"]
target = "Offer"

In [25]:
# Target Variable
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lasso_model = Lasso(alpha=1.0)
lasso_model.fit(X_train, y_train)

y_pred = lasso_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(mae, r2)

25718.969489885905 0.7582831053796442


glorious .9